
#            TP Identification de paramètres d'un robot RRR par techniques des moindres carrées #



Soit le robot RRR correspondant à la figure suivante:

![alternative text](robot-rrr3-sol.png)

- La configuration du robot est défini par le vecteur $q = (q_1, q_2, q_3)^t$.

- La situation de l'outil est défini par le vecteur $X = (x, y, z)^t$ .

## Objectif
L'objectif est d'identifier certains paramètres géométriques du robot plan RRR.

A l'aide d'une caméra, on a mesuré le vecteur $X$ pour un ensemble de configurations $q$. On considère que toutes les mesures sont exactes (pas d'erreurs liées au système de mesure).

Les résultats se trouvent dans les fichier *mesuresX.dat*. Chaque ligne comporte une mesure avec la convention suivante pour les 6 valeurs:

$q_1$, $q_2$, $q_3$, x, y, z sur chaque ligne (format %12.6f pour chaque valeur). Les longueurs sont exprimés en mètre.

- A: Dans un premier temps on veut identifier les longueurs des liaisons $a$, $b$, $c$ du robot RRR en considérant que les valeurs des angles $q_i$ sont exactes.

Fichier de mesure: *mesures1.dat*


## Travail

Modéliser le problème comme un problème d'optimisation

Implémenter votre problème et le résoudre 


In [99]:
#################################################   
# Import 
from matplotlib import cm
import matplotlib.pyplot as plt
import numpy as np
import scipy
import time
from random import *
from numpy.linalg import inv,cond
from scipy.optimize import root
from scipy.optimize import least_squares

In [109]:
#################################################   
# Calcul du MGD du robot RRR
# INPUT:  q = vecteur de configuration (deg, deg, deg)
# OUTPUT: Xc = vecteur de situation = position3D = (x,y,z) 

def mgd(qdeg,a,b,c):
    qrad=np.deg2rad(qdeg)
    # introduction d'un eerreur su run qi
    # erreur= 5 (en degré)
    # erg = np.deg2rad(erreur)
    # qrad[i] = qi + erg
    c1= np.cos(qrad[0])
    s1=np.sin(qrad[0])
    c23= np.cos(qrad[2]+qrad[1])
    s23= np.sin(qrad[2]+qrad[1])
    c2=np.cos(qrad[1])
    s2=np.sin(qrad[1])

    A=[ [c1*c2, c1*c23, 0],
        [s1*c2, s1*c23, 0],
        [s2   , s23   , 1]]
    # A=np.ndarray(mat)
    x= a*c1*c2 + b*c1*c23 
    y= a*s1*c2 + b*s1*c23 
    z= a*s2 + b*s23 + c 
    Xd=[x,y,z]
    return (Xd,A)

#

In [117]:
# Lecture mesures de fichier

with open("mesuresExactes.dat", "r") as f:
    line= list()

    donnees= list()
    xdata= list()
    qdata= list()
    q1data= list()
    q2data= list()
    q3data= list()
    for line in f:
        if "#" in line:
            # on saute la ligne
            continue
        data = line.split()
        donnees.append((float(data[0]), float(data[1]),float(data[2]), float(data[3]), float(data[4]),float(data[5])))
        xdata.append([float(data[3]), float(data[4]),float(data[5])])
        qdata.append([float(data[0]), float(data[1]),float(data[2])])
        q1data.append(float(data[0]))
        q2data.append(float(data[1]))
        q3data.append(float(data[2]))
#donnees
#xdata


In [123]:

# print(qdata)
a,b,c=1,1,1

B=[]
A = []
nb_val = 200 #len(qdata)

for i in range(200):
    a1=mgd(qdata[i],a,b,c)[1]
    for y in a1:
        A.append(y)
    b1 = xdata[i]
    for h in b1:
        B.append(h)

A=np.array(A)
B=np.array(B)
print(A.shape)
print(np.shape(B))

X = np.dot(np.dot(np.linalg.inv(np.dot(A.transpose(),A)) , A.transpose() ), B)
print(X)

(600, 3)
(600,)
[3.99999993 2.5        3.29999997]


### Test du MGD après identification

Vérifier que votre identification des longueurs $a$, $b$ et $c$ est correcte.

In [119]:
a,b,c = 4,2.5,3.3

for i in range(0,len(qdata)):
    err=[]
    mesure = mgd(qdata[i],a,b,c)[0]
    for y in range(0,len(mesure)) :
        err.append(mesure[y] - xdata[i][y])
    print(np.dot(err,err))

2.027493482568568e-13
1.1626197976974665e-13
5.1008426440298355e-14
2.0710666749290742e-13
2.745886354630821e-13
3.330223709723222e-13
3.7009720473596283e-13
3.8736627467737024e-13
3.868957659605574e-13
4.048508667459166e-13
4.8306137403840616e-14
3.219329886362999e-13
2.2201338541936991e-13
1.706412984035495e-14
5.945770258023725e-14
5.215574666052852e-13
2.967266698726411e-13
4.790581473580729e-13
2.6011456335653946e-13
1.9834551993470525e-13
4.353613935442999e-13
1.55248737294835e-14
4.353613935335457e-13
1.9834552001964408e-13
3.067587439223126e-14
4.790581473580729e-13
2.967266694829562e-13
5.215574658856969e-13
5.945770297708274e-14
1.7064129702351354e-14
4.7139022511941787e-14
3.219329886122215e-13
4.830613746826052e-14
4.04850865694546e-13
3.868957655355729e-13
3.873662733615765e-13
2.2866672160118e-13
8.601602772935097e-14
2.076247435044642e-14
2.3302403894418486e-13
1.2409164386922182e-13
2.2670326309330125e-13
1.4567013305717638e-13
1.6293920327992274e-13
6.076415776548214e-

## Utilisation de scipy optimize

Retrouver vos résultats en utilisant directement les fonctions de scipy optimize

In [126]:
from scipy.optimize import nnls

nnls(A,B)


(array([3.99999993, 2.5       , 3.29999997]), 6.851040833177421e-06)

En synthèse, cette expérience d'identification des paramètres d'un robot RRR par la technique des moindres carrés a fourni des enseignements précieux. En se penchant sur les mesures du vecteur (X) pour diverses configurations (q), l'objectif était de déterminer les longueurs des liaisons du robot. La modélisation du problème en tant que problème d'optimisation, sans explicitement calculer la jacobienne, s'est avérée une approche efficace. Les résultats ont souligné l'importance du nombre de valeurs dans le fichier de données, avec une amélioration notable de la précision en augmentant cette quantité, atteignant une remarquable erreur résiduelle de (10^{-7}). 
En comparant avec les résultats de la bibliothèque scipy.optimize, notre approche a montré sa validité, soulignant l'importance d'une modélisation précise et d'un ensemble de données étendu pour une identification précise des paramètres du robot.

